# Building Scalable Models in PySpark - Exercises with answers

## Exercise 1

#### Task 1
##### Initialize SparkContext
##### Create a RDD called ``RDD_numbers`` from a list with elements 10,20,30,40
##### Retrieve all the elements from ``RDD_numbers`` 

#### Result:

In [0]:
sc

Out[1]:

SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
numbers = [10, 20, 30, 40]
RDD_numbers = sc.parallelize(numbers)
RDD_numbers.collect()

Out[2]: [10, 20, 30, 40]

#### Task 2
##### Create a RDD called ``RDD_words`` using the words "big data", "python", "dataframe", "spark", "rdd", "spark module", "data science"
##### Retrieve all the elements from ``RDD_words``

#### Result:

In [0]:
RDD_words = sc.parallelize (["big data", "python", "dataframe", "spark", "rdd", "spark module", "data science"])
RDD_words.collect()

Out[3]: ['big data',
 'python',
 'dataframe',
 'spark',
 'rdd',
 'spark module',
 'data science']

#### Task 3
##### Create a new RDD called ``data_filter`` from the RDD created in Question 2 that contains the word ``data``
##### Retrieve all the elements from ``data_filter``

#### Result:

In [0]:
data_filter = RDD_words.filter(lambda x: 'data' in x)
data_filter.collect()

Out[4]: ['big data', 'dataframe', 'data science']

#### Task 4
##### Create a new RDD called ``spark_filter`` from the RDD created in Question 2 that contains the word ``spark``
##### Retrieve all the elements from ``spark_filter``

#### Result:

In [0]:
spark_filter = RDD_words.filter(lambda x: 'spark' in x)
spark_filter.collect()

Out[5]: ['spark', 'spark module']

#### Task 5
##### Combine `data_filter` and `spark_filter` in an RDD called `filteredwords_ex` by taking the union
##### Retrieve all the elements from **filteredwords_ex**

#### Result:

In [0]:
filteredwords_ex = data_filter.union(spark_filter)
filteredwords_ex.collect()

Out[6]: ['big data', 'dataframe', 'data science', 'spark', 'spark module']

#### Task 6
##### Let's perform some RDD actions
##### Count the number of elements present in ``RDD_words``
##### Retrieve the ``first`` element from ``RDD_words``

##### Retrieve the ``first two`` elements from ``RDD_numbers``

#### Result:

In [0]:
RDD_words.count()

Out[7]: 7

In [0]:
RDD_words.first()

Out[8]: 'big data'

In [0]:
RDD_numbers.take(2)

Out[9]: [10, 20]

## Exercise 2

#### Task 1
##### Check if the Spark session has been initialized in Databricks
##### Import all the packages required for this module
##### Create a DataFrame called `bank_data` from the file `bank_marketing.csv`. Choose `option("inferSchema", "true")` and `option("header", "true")`
##### Print the DataFrame``bank_data`` and its schema

#### Result:

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
spark

Out[11]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
bank_data = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("/FileStore/tables/bank_marketing.csv")

In [0]:
bank_data

Out[52]: DataFrame[age: int, job: string, marital: string, education: string, default: string, housing: string, loan: string, contact: string, month: string, day_of_week: string, duration: int, campaign: int, pdays: int, previous: int, poutcome: string, emp_var_rate: double, cons_price_idx: double, cons_conf_idx: double, euribor3m: double, nr_employed: int, y: string]

In [0]:
bank_data.printSchema()

root
-- age: integer (nullable = true)
-- job: string (nullable = true)
-- marital: string (nullable = true)
-- education: string (nullable = true)
-- default: string (nullable = true)
-- housing: string (nullable = true)
-- loan: string (nullable = true)
-- contact: string (nullable = true)
-- month: string (nullable = true)
-- day_of_week: string (nullable = true)
-- duration: integer (nullable = true)
-- campaign: integer (nullable = true)
-- pdays: integer (nullable = true)
-- previous: integer (nullable = true)
-- poutcome: string (nullable = true)
-- emp_var_rate: double (nullable = true)
-- cons_price_idx: double (nullable = true)
-- cons_conf_idx: double (nullable = true)
-- euribor3m: double (nullable = true)
-- nr_employed: integer (nullable = true)
-- y: string (nullable = true)

#### Task 2
##### Count the number of observations in the dataframe `bank_data`
##### Select columns `age, job` from the dataframe `bank_data` and display the first 10 rows
##### Select columns ``age, duration, campaign, education`` from the dataframe ``bank_data`` and print the descriptive statistics

#### Result:

In [0]:
bank_data.count()

Out[54]: 41188

In [0]:
bank_data.select("age","job").show(10)

+---+-----------+
age| job|
+---+-----------+
 56| housemaid|
 57| services|
 37| services|
 40| admin.|
 56| services|
 45| services|
 59| admin.|
 41|blue-collar|
 24| technician|
 25| services|
+---+-----------+
only showing top 10 rows

In [0]:
bank_data.select("age", "duration","campaign", "education").describe().show()

+-------+------------------+------------------+------------------+---------+
summary| age| duration| campaign|education|
+-------+------------------+------------------+------------------+---------+
 count| 41188| 41188| 41188| 41188|
 mean| 40.02406040594348| 258.2850101971448| 2.567592502670681| null|
 stddev|10.421249980934045|259.27924883646506|2.7700135429023245| null|
 min| 17| 0| 1| basic.4y|
 max| 98| 4918| 56| unknown|
+-------+------------------+------------------+------------------+---------+

#### Task 3
##### Use the aggregate function to find the total sum of `age` 
##### Find the sum of `age` for each `education` group
##### Find the number of distinct values from the column `education`
##### Calculate the pairwise frequency of categorical columns `job and marital`

#### Result:

In [0]:
bank_data.agg({'age':'sum'}).show()

+--------+
sum(age)|
+--------+
 1648511|
+--------+

In [0]:
bank_data.groupby("education").sum("age").show()

+-------------------+--------+
 education|sum(age)|
+-------------------+--------+
 high.school| 361553|
 unknown| 75266|
 basic.6y| 92709|
professional.course| 210140|
 university.degree| 473082|
 illiterate| 873|
 basic.4y| 198763|
 basic.9y| 236125|
+-------------------+--------+

In [0]:
bank_data.select('education').distinct().count()

Out[60]: 8

In [0]:
bank_data.crosstab('job', 'marital').show()

+-------------+--------+-------+------+-------+
 job_marital|divorced|married|single|unknown|
+-------------+--------+-------+------+-------+
 housemaid| 161| 777| 119| 3|
 services| 532| 2294| 1137| 6|
self-employed| 133| 904| 379| 5|
 student| 9| 41| 824| 1|
 retired| 348| 1274| 93| 5|
 unknown| 13| 234| 74| 9|
 admin.| 1280| 5253| 3875| 14|
 blue-collar| 728| 6687| 1825| 14|
 technician| 774| 3670| 2287| 12|
 entrepreneur| 179| 1071| 203| 3|
 management| 331| 2089| 501| 3|
 unemployed| 124| 634| 251| 5|
+-------------+--------+-------+------+-------+

#### Task 4
##### Filter the column `age` for values less than 30 and select columns  `age, job, marital, education`
##### Sort the resulting subset by ``age`` in descending order and display the first 10 rows

#### Result:

In [0]:
bank_data.filter("age < 30").select("age","job","marital","education").sort('age', ascending=False).show(10)

+---+-------------+-------+-----------------+
age| job|marital| education|
+---+-------------+-------+-----------------+
 29| technician| single|university.degree|
 29|self-employed|married|university.degree|
 29|self-employed|married|university.degree|
 29| admin.| single|university.degree|
 29| technician| single|university.degree|
 29|self-employed|married|university.degree|
 29| admin.|married|university.degree|
 29| admin.| single|university.degree|
 29| admin.|unknown|university.degree|
 29|self-employed|married|university.degree|
+---+-------------+-------+-----------------+
only showing top 10 rows

## Exercise 3

#### Task 1
##### Create a SQL table ``bank_data_sql_table`` from the dataframe ``bank_data`` . This table will be used to perform SQL operations
##### Use a SQL statement to subset the columns `age, job, marital, education` from the table ``bank_data_sql_table`` and save it as `bank_subset`. Display the top 5 rows from ``bank_subset``
##### Count the number of records from the ``bank_data_sql_table``. Name the columns as `Number_of_Records`

##### Filter data with `education` starting with 'basic'. Find the average `duration` grouped by `education` and `marital` 

#### Result:

In [0]:
bank_data.createOrReplaceTempView("bank_data_sql_table")

In [0]:
bank_subset = spark.sql("SELECT age,job,marital,education FROM bank_data_sql_table")
bank_subset.show(5)

+---+---------+-------+-----------+
age| job|marital| education|
+---+---------+-------+-----------+
 56|housemaid|married| basic.4y|
 57| services|married|high.school|
 37| services|married|high.school|
 40| admin.|married| basic.6y|
 56| services|married|high.school|
+---+---------+-------+-----------+
only showing top 5 rows

In [0]:
spark.sql("SELECT count(*) as Number_of_Records FROM bank_data_sql_table").show()

+-----------------+
Number_of_Records|
+-----------------+
 41188|
+-----------------+

In [0]:
spark.sql("SELECT marital,education,avg(duration) FROM bank_data_sql_table where education like 'basic%' group by marital,education").show()

+--------+---------+------------------+
 marital|education| avg(duration)|
+--------+---------+------------------+
 single| basic.9y| 260.5387537993921|
 married| basic.4y|265.40055762081784|
 unknown| basic.9y| 287.125|
divorced| basic.4y| 258.7321063394683|
divorced| basic.9y| 256.7008849557522|
 married| basic.9y| 262.1046679499519|
divorced| basic.6y| 278.1098901098901|
 single| basic.6y| 285.8783382789318|
 married| basic.6y| 258.320882852292|
 unknown| basic.6y| 438.0|
 unknown| basic.4y| 542.3333333333334|
 single| basic.4y| 263.6512141280353|
+--------+---------+------------------+

#### Task 2

##### Check for NAs for all the columns in the DataFrame `bank_data` and display the count of NAs in each column
##### Identify the columns with NAs
##### Drop the NAs from the data and save it in the same DataFrame `bank_data`
##### Verify whether all the NAs have been dropped from the DataFrame `bank_data`

#### Result:

In [0]:
bank_data.select([count(when(isnull(c), c)).alias(c) for c in bank_data.columns]).show()

+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
age|job|marital|education|default|housing|loan|contact|month|day_of_week|duration|campaign|pdays|previous|poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed| y|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+

Column 'pdays' has one NULL value

In [0]:
bank_data = bank_data.dropna()
bank_data.select([count(when(isnull(c), c)).alias(c) for c in bank_data.columns]).show()

+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
age|job|marital|education|default|housing|loan|contact|month|day_of_week|duration|campaign|pdays|previous|poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed| y|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+

#### Task 3
##### Perform the String Indexing opertion on the column `job`. Name the output column as `jobindex` and save the DataFrame as `bank_indexed`
##### Display the first 10 rows of columns `job` and `jobindex` from `bank_indexed`
##### Perform One-hot encoding on the column `jobindex` of the DataFrame `bank_indexed`. Name the output column as `jobVec1` and save the DataFrame as `bank_encoded`
##### Display the first 10 rows of columns `job`, `jobindex` and `jobVec1` from `bank_encoded`

#### Result:

In [0]:
indexer = StringIndexer(inputCol="job", outputCol="job" + "index")
bank_indexed = indexer.fit(bank_data).transform(bank_data)
bank_indexed.select("job","jobindex").show(10)

+-----------+--------+
 job|jobindex|
+-----------+--------+
 services| 3.0|
 services| 3.0|
 admin.| 0.0|
 services| 3.0|
 services| 3.0|
 admin.| 0.0|
blue-collar| 1.0|
 technician| 2.0|
 services| 3.0|
blue-collar| 1.0|
+-----------+--------+
only showing top 10 rows

In [0]:
encoder = OneHotEncoder(inputCols=["jobindex"],
                        outputCols=["jobVec1"])
bank_encoded = encoder.fit(bank_indexed).transform(bank_indexed)
bank_encoded.select("job","jobindex","jobVec1").show(10)

+-----------+--------+--------------+
 job|jobindex| jobVec1|
+-----------+--------+--------------+
 services| 3.0|(11,[3],[1.0])|
 services| 3.0|(11,[3],[1.0])|
 admin.| 0.0|(11,[0],[1.0])|
 services| 3.0|(11,[3],[1.0])|
 services| 3.0|(11,[3],[1.0])|
 admin.| 0.0|(11,[0],[1.0])|
blue-collar| 1.0|(11,[1],[1.0])|
 technician| 2.0|(11,[2],[1.0])|
 services| 3.0|(11,[3],[1.0])|
blue-collar| 1.0|(11,[1],[1.0])|
+-----------+--------+--------------+
only showing top 10 rows

#### Task 4
##### Use VectorAssembler to combine the columns `previous` and `emp_var_rate` into column `vector`. Peform this operation on the
##### `bank_encoded` DataFrame and save the new DataFrame as `bank_assembler_model`
##### Display the first 10 rows of columns `previous`, `emp_var_rate` and `vector` from the `bank_assembler_model` DataFrame

#### Result:

In [0]:
## VectorAssembler to combine all the feature columns into a single vector column
assembler = VectorAssembler(inputCols=["previous","emp_var_rate"], outputCol="vector")
bank_assembler_model = assembler.transform(bank_encoded)
bank_assembler_model.select("previous","emp_var_rate","vector").show(10)

+--------+------------+---------+
previous|emp_var_rate| vector|
+--------+------------+---------+
 0| 1.1|[0.0,1.1]|
 0| 1.1|[0.0,1.1]|
 0| 1.1|[0.0,1.1]|
 0| 1.1|[0.0,1.1]|
 0| 1.1|[0.0,1.1]|
 0| 1.1|[0.0,1.1]|
 0| 1.1|[0.0,1.1]|
 0| 1.1|[0.0,1.1]|
 0| 1.1|[0.0,1.1]|
 0| 1.1|[0.0,1.1]|
+--------+------------+---------+
only showing top 10 rows

#### Task 5 - Data Preparation for Logistic Regression

##### List all the categorical variables in `categorical_columns_ex` and create a new list `pipeline_stages_ex` for pipelining all the stages.
##### Perform string indexing on all the `categorical columns`. Add suffix `Index` for all the output columns of StringIndexer 
##### Perform One-hot encoding on all the `String Indexer output columns`. Add suffix `One_hot_vec` for all the output columns of One-hot encoder
##### Append all the operations to `pipeline_stages_ex`
##### Perform string indexing on the target variable `y`. Name the output column as `y_index` and append this operation to `pipeline_stages_ex`

#### Result:

In [0]:
categorical_columns_ex = ["job","marital","education","default","housing","loan","contact","month","day_of_week","poutcome"]
pipeline_stages_ex = []

for column in categorical_columns_ex:
    stringIndexer = StringIndexer(inputCol = column, outputCol = column + 'Index')
    one_hot_encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[column + "One_hot_vec"])
    pipeline_stages_ex += [stringIndexer, one_hot_encoder]

In [0]:
label_index = StringIndexer(inputCol = 'y', outputCol = 'y_index')
pipeline_stages_ex += [label_index]

#### Task 6 - Data Preparation for Logistic Regression

##### List all the numeric variables in `numeric_columns_ex`. 
##### Append `numeric_columns_ex` with the One-hot encoder output columns. Name the appended list as `inputs_ex`
##### Apply VectorAssembler on `inputs_ex` and name the output column as `features`.
##### Append the assembler operations to `pipeline_stages_ex`
##### Perform stringindexing on the target variable `y`. Name the output column as `y_index` and append this operation to `pipeline_stages_ex`

#### Result:

In [0]:
numeric_columns_ex = ["age","duration","campaign","pdays","previous","emp_var_rate","cons_price_idx","cons_conf_idx","euribor3m","nr_employed"]
inputs_ex = [c + "One_hot_vec" for c in categorical_columns_ex] + numeric_columns_ex
assembler = VectorAssembler(inputCols=inputs_ex, outputCol="features")
pipeline_stages_ex += [assembler]

#### Task 7 - Data Preparation for Logistic Regression

##### Create a pipeline `pipeline_ex` for the stages created
##### Fit the pipeline `pipeline_ex` on the DataFrame `bank_data` and name the model `pipelineModel_ex`
##### Transform the `bank_data` DataFrame using `pipelineModel_ex` and save it as `bank_data_logistic`

#### Result:

In [0]:
pipeline_ex = Pipeline(stages = pipeline_stages_ex)
pipelineModel_ex = pipeline_ex.fit(bank_data)
bank_data_logistic = pipelineModel_ex.transform(bank_data)

#### Task 8 - Data Preparation for Logistic Regression

##### Split `bank_data_logistic` to train and test in the ratio of 70:30. Save the DataFrames as `train` and `test`. Set the seed value to 2
##### Count the number of observations in train and test dataframes

#### Result:

In [0]:
train, test = bank_data_logistic.randomSplit([0.7, 0.3], seed = 2)

In [0]:
train.count()

Out[78]: 28883

In [0]:
test.count()

Out[79]: 12304

## Exercise 4

#### Task 1

##### Apply `Logistic Regression` with input columns as `features` and target variable as `y_index` and save the model as `lr_ex`
##### Apply the model `lr_ex` on train
##### Print the model `lr_ex`

#### Result:

In [0]:
lr_ex = LogisticRegression(featuresCol = 'features', labelCol = 'y_index', maxIter=10)
lrModel = lr_ex.fit(train)

In [0]:
lrModel

Out[81]: LogisticRegressionModel: uid=LogisticRegression_e98a63dbfae3, numClasses=2, numFeatures=53

#### Task 2

##### Predict the target variable on the test data using the model `lrModel`. Save the predictions as `predictions_ex`
##### Select `y_index, rawPrediction, prediction, probability` from the `predictions_ex` DataFrame

#### Result:

In [0]:
predictions_ex = lrModel.transform(test)
predictions_ex.select('y_index', 'rawPrediction', 'prediction', 'probability').show(10)

+-------+--------------------+----------+--------------------+
y_index| rawPrediction|prediction| probability|
+-------+--------------------+----------+--------------------+
 1.0|[2.05355528828481...| 0.0|[0.88630636818059...|
 1.0|[2.05411529015604...| 0.0|[0.88636278590104...|
 1.0|[2.05459712105922...| 0.0|[0.88641130870543...|
 1.0|[2.05173498039814...| 0.0|[0.88612281148028...|
 1.0|[2.05183789342197...| 0.0|[0.8861331959359,...|
 1.0|[2.05351307841744...| 0.0|[0.88630211473307...|
 0.0|[2.06852627112409...| 0.0|[0.88780625262087...|
 0.0|[2.06313612669408...| 0.0|[0.88726823689279...|
 0.0|[2.06073728486301...| 0.0|[0.88702807379101...|
 0.0|[2.07082914794176...| 0.0|[0.88803542891229...|
+-------+--------------------+----------+--------------------+
only showing top 10 rows

#### Task 3

##### Calculate the `Accuracy` on the test predictions and print it
##### Calculate the `Area Under ROC` on the test predictions and print it

#### Result:

In [0]:
accuracy = predictions_ex.filter(predictions_ex.y_index == predictions_ex.prediction).count() / float(predictions_ex.count())
print("Accuracy : ", accuracy)

Accuracy : 0.8870286085825748

In [0]:
evaluator = BinaryClassificationEvaluator(labelCol = "y_index")
print('Test Area Under ROC', evaluator.evaluate(predictions_ex))

Test Area Under ROC 0.8713732477459465